## Populate the Job Posting Ontology

In [1]:
# required libraries
import pandas as pd
import os
from pathlib import Path
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace, BNode, OWL, RDFS
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD
# CHECK DATE 
import datetime

In [2]:
# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
print(path)
jobPostingUrl = 'data/job_posting.csv'
print(jobPostingUrl)

companiesUrl = path + '/GraphDB/data/company_details/companies.csv'
company_industriesUrl = path + '/GraphDB/data/company_details/company_industries.csv'
company_specialitiesUrl = path + '/GraphDB/data/company_details/company_specialities.csv'
specialitiesUrl = path + '/GraphDB/data/company_details/specialities.csv'
employee_countsUrl = path + '/GraphDB/data/company_details/employee_counts.csv'
c = path + '/GraphDB/data/company_details/specialities.csv'

benefitsUrl = path + '/GraphDB/data/job_details/benefits.csv'
job_industriesUrl = path + '/GraphDB/data/job_details/job_industries.csv'
job_skillsUrl = path + '/GraphDB/data/job_details/job_skills.csv'
salariesUrl = path + '/GraphDB/data/job_details/salaries.csv'

industriesUrl = path + '/GraphDB/data/maps/industries.csv'
skillsUrl = path + '/GraphDB/data/maps/skills.csv'


# country codes
countriesURL = path + '/GraphDB/data/countries/all.csv'

# saving folder
savePath =  path + '/data/rdf/linkedinDB/' 

c:\Users\Navid\Documents\GitHub
data/job_posting.csv


In [3]:
# Construct the country and the movie ontology namespaces not known by RDFlib
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
LNJP = Namespace("http://www.dei.unipd.it/database2/LinkedinJobPosting#")
SKOS = Namespace("https://www.w3.org/2009/08/skos-reference/skos-owl1-dl.rdf")


## Job Posting

In [4]:
# Load the CSV files in memory
jobPosting = pd.read_csv(jobPostingUrl, sep=',', index_col='job_id')
#print(jobPosting)

In [5]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)


In [6]:
# Iterate over the job postings DataFrame
for index, row in jobPosting.iterrows():
    # Create the JobPosting node with a URI
    jobPostingId = "job_posting" + str(index)
    JobPosting = URIRef(LNJP[jobPostingId])
    g.add((JobPosting, RDF.type, LNJP.JobPosting))


    #---- for check ---

    # TransfermarktProfile = URIRef(row['url'])
    
    # # Add triples using store's add() method.
    # g.add((Player, RDF.type, SO.Player))
    # g.add((TransfermarktProfile, RDF.type, SO.TransfermarktProfile))
    # g.add((TransfermarktProfile, SO['isAbout'], Player))

    #------------------

    # Add data properties
    g.add((JobPosting, LNJP.job_id, Literal(index, datatype=XSD.int)))
    g.add((JobPosting, LNJP.title, Literal(row['title'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.description, Literal(row['description'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.max_salary, Literal(row['max_salary'], datatype=XSD.int)))
    g.add((JobPosting, LNJP.med_salary, Literal(row['med_salary'], datatype=XSD.int)))
    g.add((JobPosting, LNJP.min_salary, Literal(row['min_salary'], datatype=XSD.int)))
    g.add((JobPosting, LNJP.pay_period, Literal(row['pay_period'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.location, Literal(row['location'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.applies, Literal(row['applies'], datatype=XSD.int)))
    g.add((JobPosting, LNJP.original_listed_time, Literal(row['original_listed_time'], datatype=XSD.dateTime)))
    g.add((JobPosting, LNJP.remote_allowed, Literal(row['remote_allowed'], datatype=XSD.int)))
    g.add((JobPosting, LNJP.views, Literal(row['views'], datatype=XSD.int)))
    g.add((JobPosting, LNJP.job_posting_url, Literal(row['job_posting_url'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.application_url, Literal(row['application_url'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.application_type, Literal(row['application_type'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.expiry, Literal(row['expiry'], datatype=XSD.dateTime)))
    g.add((JobPosting, LNJP.closed_time, Literal(row['closed_time'], datatype=XSD.dateTime)))
    g.add((JobPosting, LNJP.formatted_experience_level, Literal(row['formatted_experience_level'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.skills_desc, Literal(row['skills_desc'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.listed_time, Literal(row['listed_time'], datatype=XSD.dateTime)))
    g.add((JobPosting, LNJP.posting_domain, Literal(row['posting_domain'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.sponsored, Literal(row['sponsored'], datatype=XSD.int)))
    g.add((JobPosting, LNJP.work_type, Literal(row['work_type'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.currency, Literal(row['currency'], datatype=XSD.string)))
    g.add((JobPosting, LNJP.compensation_type, Literal(row['compensation_type'], datatype=XSD.string)))

    Company = URIRef(LNJP["company_" + str(row['company_id'])])
    g.add((JobPosting, LNJP['hasCompany'], Company))
# terminated
# jobSkillsAbr = "skill_" + str(row['abr']) # You should replace this with the actual ID or URI
# JobSkills = URIRef(LNJP[jobSkillsAbr])
# print(JobSkills)


# Serialize the RDF graph to Turtle format
turtle_file_path = 'job_postings.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()
    
    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")


--- saving serialization ---
RDF data exported to job_postings.ttl
Serialization saved successfully.


## Skills

In [7]:
# Load the CSV files in memory
skills = pd.read_csv(skillsUrl, sep=',', index_col='skill_abr', keep_default_na=False, na_values=['_'])
assert not skills.empty, "Skills DataFrame is empty. Check the CSV file or URL."

# Print a message if the test passes
print("Skills DataFrame read successfully.")
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

Skills DataFrame read successfully.


In [8]:
# Assuming you have a DataFrame called 'skillsDataFrame' with columns 'skill_abr' and 'skill_name'
# Iterate over all rows in the skills DataFrame
for skill_abr, row in skills.iterrows():
    # Create Skill node with a URI
    skillId = "skill_" + str(skill_abr)
    Skill = URIRef(LNJP[skillId])
    g.add((Skill, RDF.type, SKOS.Concept))
    g.add((Skill, RDF.type, LNJP.Skill))

    # Add skill information as data properties
    g.add((Skill, LNJP.skill_abr, Literal(skill_abr, datatype=XSD.string)))
    g.add((Skill, LNJP.skill_name, Literal(row['skill_name'], datatype=XSD.string)))

    if(row['broader_concept']):
        broaderSkillId = "skill_" + str(row['broader_concept'])
        broaderSkill = URIRef(LNJP[broaderSkillId])
        g.add((Skill, SKOS.broader, broaderSkill))
    if(row['narrower_concept']):
        narrowerSkillId = "skill_" + str(row['narrower_concept'])
        narrowerSkill = URIRef(LNJP[narrowerSkillId])
        g.add((Skill, SKOS.narrower, narrowerSkill))
        
    if(row['related_concept']):
        relatedSkillId = "skill_" + str(row['related_concept'])
        relatedSkill = URIRef(LNJP[relatedSkillId])
        g.add((Skill, SKOS.related, relatedSkill))
    

# Serialize the RDF graph to Turtle format
turtle_file_path = 'skills.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")


--- saving serialization ---
RDF data exported to skills.ttl
Serialization saved successfully.


## JOB_SKILLS JOIN

In [9]:
# Load the CSV files in memory
job_skills = pd.read_csv(job_skillsUrl, sep=',', index_col='job_id')
assert not job_skills.empty, "Skills DataFrame is empty. Check the CSV file or URL."

# Print a message if the test passes
print("Skills DataFrame read successfully.")
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

Skills DataFrame read successfully.


In [10]:
# Assuming you have a DataFrame called 'skillsDataFrame' with columns 'skill_abr' and 'skill_name'
# Iterate over all rows in the skills DataFrame
for index, row in job_skills.iterrows():
    # Create Skill node with a URI
    # skillId = "skill_" + str(skill_abr)
    
    jobPostingId = "job_posting" + str(index)
    JobPosting = URIRef(LNJP[jobPostingId])
    Skill = URIRef(LNJP[row['skill_abr']])
    g.add((JobPosting, LNJP['hasSkill'], Skill))
    



# Serialize the RDF graph to Turtle format
turtle_file_path = 'job_skills_join.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")

--- saving serialization ---
RDF data exported to job_skills_join.ttl
Serialization saved successfully.


## Industry

In [11]:
# Load the CSV files in memory
industries = pd.read_csv(industriesUrl, sep=',', index_col='industry_id', keep_default_na=False, na_values=['_'])
assert not industries.empty, "Industries DataFrame is empty. Check the CSV file or URL."

# Print a message if the test passes
print("Industries DataFrame read successfully.")
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

Industries DataFrame read successfully.


In [12]:
# Assuming you have a DataFrame called 'industiesDataFrom' with columns 'industry_id' and 'industry_name'
# Iterate over all rows in the industries DataFrame
for index, row in industries.iterrows():
    # Create Skill node with a URI
    industryId = "industry_" + str(index)
    Industry = URIRef(LNJP[industryId])
    g.add((Industry, RDF.type, LNJP.Industry))

    # Add skill information as data properties
    g.add((Industry, LNJP['industry_id'], Literal(index, datatype=XSD.string)))
    g.add((Industry, LNJP['industry_name'], Literal(row['industry_name'], datatype=XSD.string)))
    

# Serialize the RDF graph to Turtle format
turtle_file_path = 'industries.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")


--- saving serialization ---
RDF data exported to industries.ttl
Serialization saved successfully.


## Job_Industries join

In [13]:
# Load the CSV files in memory
job_industries = pd.read_csv(job_industriesUrl, sep=',', index_col='job_id')
assert not job_industries.empty, "Skills DataFrame is empty. Check the CSV file or URL."

# Print a message if the test passes
print("Skills DataFrame read successfully.")
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

Skills DataFrame read successfully.


In [14]:
# Assuming you have a DataFrame called 'skillsDataFrame' with columns 'skill_abr' and 'skill_name'
# Iterate over all rows in the skills DataFrame
for index, row in job_industries.iterrows():
    # Create Skill node with a URI
    # skillId = "skill_" + str(skill_abr)
    
    jobPosting_id = "job_posting" + str(index)
    JobPosting = URIRef(LNJP[jobPosting_id])
    Industry = URIRef(LNJP["industry_" + str(row['industry_id'])])
    g.add((JobPosting, LNJP['hasIndustryType'], Industry))
    



# Serialize the RDF graph to Turtle format
turtle_file_path = 'job_industries_join.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")

--- saving serialization ---
RDF data exported to job_industries_join.ttl
Serialization saved successfully.


## Company

In [15]:
# Load the CSV files in memory
companies = pd.read_csv(companiesUrl, sep=',', index_col='company_id', keep_default_na=False, na_values=['_'])
assert not companies.empty, "Companies DataFrame is empty. Check the CSV file or URL."

# Print a message if the test passes
print("Companies DataFrame read successfully.")
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

Companies DataFrame read successfully.


In [5]:
# Define classes and properties
largeCompany = LNJP.largeCompany
company_size_property = LNJP['company_size']

BN0 = BNode()
BN1 = BNode()
BN2 = BNode()
BN3 = BNode()
BN4 = BNode()
BN5 = BNode()
BN6 = BNode()
BN7 = BNode()
BN8 = BNode()

# Define the class and its equivalent class
g.add((largeCompany, RDF.type, OWL.Class))
g.add((largeCompany, RDFS.subClassOf, LNJP.Company))

g.add((largeCompany, OWL.equivalentClass, BN0))

g.add((BN0, RDF.type, OWL.Class))
g.add((BN0, OWL.intersectionOf, BN1))

g.add((BN1, RDF.type, RDF.List))
g.add((BN1, RDF.first, LNJP.Company))
g.add((BN1, RDF.rest, BN2))

g.add((BN2, RDF.type, RDF.List))
g.add((BN2, RDF.first, BN3))
g.add((BN2, RDF.rest, RDF.nil))

g.add((BN3, RDF.type, OWL.Restriction))
g.add((BN3, OWL.onProperty, company_size_property))
g.add((BN3, OWL.someValuesFrom, BN4))

g.add((BN4, RDF.type, RDFS.Datatype))
g.add((BN4, OWL.onDatatype, XSD.integer))
g.add((BN4, OWL.withRestrictions, BN5))

g.add((BN5, RDF.type, RDF.List))
g.add((BN6, RDF.type, RDF.List))
g.add((BN7, RDF.type, RDF.List))
g.add((BN8, RDF.type, RDF.List))

g.add((BN5, RDF.first, BN6))
g.add((BN5, RDF.rest, BN7))

g.add((BN7, RDF.first, BN8))
g.add((BN7, RDF.rest, RDF.nil))

g.add((BN6, XSD.minInclusive, Literal("6", datatype=XSD.integer)))
g.add((BN8, XSD.maxInclusive, Literal("7", datatype=XSD.integer)))

# Assuming you have a DataFrame called 'companiesDataFrom' with columns 'company_id', 'name', 'description', 'company_size', 'state', 'country', 'city', 'zip_code', 'address' and 'url'
# Iterate over all rows in the companies DataFrame
for index, row in companies.iterrows():
    # Create Company node with a URI
    companyId = "company_" + str(index)
    Company = URIRef(LNJP[companyId])
    g.add((Company, RDF.type, LNJP.Company))

    # Add Company information as data properties
    g.add((Company, LNJP['company_id'], Literal(index, datatype=XSD.string)))
    g.add((Company, LNJP['name'], Literal(row['name'], datatype=XSD.string)))
    g.add((Company, LNJP['description'], Literal(row['description'], datatype=XSD.string)))
    if not pd.isnull(row['company_size']) and row['company_size'] != "":
        g.add((Company, LNJP['company_size'], Literal(row['company_size'], datatype=XSD.integer)))
    g.add((Company, LNJP['state'], Literal(row['state'], datatype=XSD.string)))
    g.add((Company, LNJP['city'], Literal(row['city'], datatype=XSD.string)))
    g.add((Company, LNJP['zip_code'], Literal(row['zip_code'], datatype=XSD.string)))
    g.add((Company, LNJP['address'], Literal(row['address'], datatype=XSD.string)))
    g.add((Company, LNJP['url'], Literal(row['url'], datatype=XSD.string)))

    Country = URIRef(CNS[row['country']])
    g.add((Company, LNJP['hasCountry'], Country))
# Serialize the RDF graph to Turtle format
turtle_file_path = 'companies.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")

--- saving serialization ---
RDF data exported to companies.ttl
Serialization saved successfully.


## Company_Industries join

In [1]:
# Load the CSV files in memory
company_industries = pd.read_csv(company_industriesUrl, sep=',', index_col='company_id', keep_default_na=False, na_values=['_'])
assert not company_industries.empty, "Skills DataFrame is empty. Check the CSV file or URL."

# Print a message if the test passes
print("Skills DataFrame read successfully.")
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

NameError: name 'pd' is not defined

In [ ]:
# Assuming you have a DataFrame called 'skillsDataFrame' with columns 'skill_abr' and 'skill_name'
# Iterate over all rows in the skills DataFrame
for index, row in company_industries.iterrows():
    # Create Skill node with a URI
    # skillId = "skill_" + str(skill_abr)
    
    company_id = "company_" + str(index)
    Company = URIRef(LNJP[company_id])
    Industry = URIRef(LNJP["industry_" + str(row['industry_id'])])
    g.add((Company, LNJP['hasIndustry'], Industry))
    



# Serialize the RDF graph to Turtle format
turtle_file_path = 'company_industries_join.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")

--- saving serialization ---
RDF data exported to company_industries_join.ttl
Serialization saved successfully.


## Person Counts

In [19]:
# Load the CSV files in memory
employee_counts = pd.read_csv(employee_countsUrl, sep=',', index_col='company_record_id')

# print(Company)

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

In [20]:
# Iterate over all rows in the companies DataFrame
for index, row in employee_counts.iterrows():
    # Create Company node with a URI
    company_record_id = "company_record_" + str(index)
    Record = URIRef(LNJP[company_record_id])
    g.add((Record, RDF.type, LNJP.Record))

    # Add Company information as data properties
    g.add((Record, LNJP['company_record_id'], Literal(index, datatype=XSD.string)))
    g.add((Record, LNJP['employee_count'], Literal(row['employee_count'], datatype=XSD.integer)))
    g.add((Record, LNJP['follower_count'], Literal(row['follower_count'], datatype=XSD.integer)))
    g.add((Record, LNJP['time_recorded'], Literal(row['time_recorded'], datatype=XSD.dateTimeStamp)))

    Company = URIRef(LNJP["company_" + str(row['company_id'])])
    g.add((Record, LNJP['IsForCompany'], Company))
# Serialize the RDF graph to Turtle format
turtle_file_path = 'employee_counts.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")

--- saving serialization ---
RDF data exported to employee_counts.ttl
Serialization saved successfully.


## Benefits

In [21]:
# Load the CSV files in memory
benefits = pd.read_csv(benefitsUrl, sep=',', index_col='benefit_id')

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

In [22]:
# Iterate over all rows in the companies DataFrame
for index, row in benefits.iterrows():
    # Create Company node with a URI
    benefit_id = "benefit_" + str(index)
    Benefit = URIRef(LNJP[benefit_id])
    g.add((Benefit, RDF.type, LNJP.Benefit))

    # Add Company information as data properties
    g.add((Benefit, LNJP['benefit_id'], Literal(index, datatype=XSD.integer)))
    g.add((Benefit, LNJP['inferred'], Literal(row['inferred'], datatype=XSD.boolean)))
    g.add((Benefit, LNJP['type'], Literal(row['type'], datatype=XSD.string)))

    JobPosting = URIRef(LNJP["job_" + str(row['job_id'])])
    g.add((Benefit, LNJP['isForJobPosting'], JobPosting))
    
# Serialize the RDF graph to Turtle format
turtle_file_path = 'benefits.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")



--- saving serialization ---
RDF data exported to benefits.ttl
Serialization saved successfully.


## Specialities

In [23]:
# Load the CSV files in memory
specialities = pd.read_csv(specialitiesUrl, sep=',', index_col='speciality_id', keep_default_na=False, na_values=['_'])

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

In [24]:
# Iterate over all rows in the companies DataFrame
for index, row in specialities.iterrows():
    # Create Company node with a URI
    speciality_id = "speciality_" + str(index)
    Speciality = URIRef(LNJP[speciality_id])
    g.add((Speciality, RDF.type, LNJP.Speciality))

    # Add Company information as data properties
    g.add((Speciality, LNJP['speciality_id'], Literal(index, datatype=XSD.integer)))
    g.add((Speciality, LNJP['speciality'], Literal(row['speciality'], datatype=XSD.string)))
# Serialize the RDF graph to Turtle format
turtle_file_path = 'speciality.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")


--- saving serialization ---
RDF data exported to speciality.ttl
Serialization saved successfully.


## Company_Specialities Join

In [25]:
# Load the CSV files in memory
company_specialities = pd.read_csv(company_specialitiesUrl, sep=',', index_col='company_id', keep_default_na=False, na_values=['_'])

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

In [26]:
# Assuming you have a DataFrame called 'skillsDataFrame' with columns 'skill_abr' and 'skill_name'
# Iterate over all rows in the skills DataFrame
for index, row in company_specialities.iterrows():
    # Create Skill node with a URI
    # skillId = "skill_" + str(skill_abr)
    
    company_id = "company_" + str(index)
    Company = URIRef(LNJP[company_id])
    Speciality = URIRef(LNJP["speciality_" + str(row['speciality_id'])])
    g.add((Company, LNJP['hasSpeciality'], Speciality))
    



# Serialize the RDF graph to Turtle format
turtle_file_path = 'company_specialities_join.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")

--- saving serialization ---
RDF data exported to company_specialities_join.ttl
Serialization saved successfully.


## Salaries

In [27]:
# Load the CSV files in memory
salaries = pd.read_csv(salariesUrl, sep=',', index_col='salary_id')

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("lnjp", LNJP)
g.bind("skos", SKOS)

In [28]:
# Iterate over all rows in the companies DataFrame
for index, row in salaries.iterrows():
    # Create Company node with a URI
    salary_id = "salary_" + str(index)
    Salary = URIRef(LNJP[salary_id])
    g.add((Salary, RDF.type, LNJP.Salary))

    # Add Company information as data properties
    g.add((Salary, LNJP['salary_id'], Literal(index, datatype=XSD.integer)))
    g.add((Salary, LNJP['max_salary'], Literal(row['max_salary'], datatype=XSD.decimal)))
    g.add((Salary, LNJP['med_salary'], Literal(row['med_salary'], datatype=XSD.decimal)))
    g.add((Salary, LNJP['min_salary'], Literal(row['min_salary'], datatype=XSD.decimal)))
    g.add((Salary, LNJP['pay_period'], Literal(row['pay_period'], datatype=XSD.string)))
    g.add((Salary, LNJP['currency'], Literal(row['currency'], datatype=XSD.string)))
    g.add((Salary, LNJP['compensation_type'], Literal(row['compensation_type'], datatype=XSD.string)))

    JobPosting = URIRef(LNJP["job_" + str(row['job_id'])])
    g.add((Salary, LNJP['isAllocatedFor'], JobPosting))
# Serialize the RDF graph to Turtle format
turtle_file_path = 'salaries.ttl'

# Check if the file already exists
if os.path.exists(turtle_file_path):
    user_input = input(f"The file '{turtle_file_path}' already exists. Do you want to rewrite it? (y/n): ").lower()

    if user_input != 'y':
        print("Serialization not saved. Exiting.")
        # You might want to add additional logic or exit the script if the user chooses not to overwrite the file.
        exit()

# If the file doesn't exist or the user chose to overwrite, proceed with saving the serialization
print("--- saving serialization ---")
g.serialize(destination=turtle_file_path, format='turtle')

print(f"RDF data exported to {turtle_file_path}")
print("Serialization saved successfully.")

--- saving serialization ---
RDF data exported to salaries.ttl
Serialization saved successfully.


## EXTRA

In [29]:
#load the country codes
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
countries = pd.read_csv(countriesURL, sep=',', index_col='name', keep_default_na=False, na_values=['_'])


In [30]:
from difflib import SequenceMatcher
import numpy as np

import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

import re

In [31]:
%%time 
#measure execution time

#iterate over the players dataframe
for index, row in players.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the player id as URI
    idU = "player"+str(index)
    Player = URIRef(SO[idU])
    # the transferMarkt profile has as URI, the URL of the profile in the website
    TransfermarktProfile = URIRef(row['url'])
    
    # Add triples using store's add() method.
    g.add((Player, RDF.type, SO.Player))
    g.add((TransfermarktProfile, RDF.type, SO.TransfermarktProfile))
    g.add((TransfermarktProfile, SO['isAbout'], Player))
    
    #process player name
    name = row['name'].split('-')

    if (len(name)>1):
        g.add((Player, SO['firstName'], Literal(name[0], datatype=XSD.string)))
        g.add((Player, SO['lastName'], Literal(name[1], datatype=XSD.string)))
    else:
        g.add((Player, SO['lastName'], Literal(name[0], datatype=XSD.string)))
        
    #there can be more than one position per player
    for pos in row['position'].split(' - '):
        g.add((Player, SO['position'], Literal(pos.lower(), datatype=XSD.string)))
    
    if not(row['club_id']==''):
        idC = "club"+str(row['club_id'])
        g.add((Player, SO['playFor'], URIRef(SO[idC])))

#iterate over the fifa dataframe
for index, row in playersFifa.iterrows():
    pname = row['short_name'].lower()
    if ('.' in pname):
        # get last name
        # in the fifa dataset we have short names as L. Messi so we delete the L. 
        # we need to check if the last name contains a space
        pname = row['short_name'].split('.')[1].lower().strip()
        if ' ' in pname:
            i = 0
            for t in pname.split(' '):
                if i == 0:
                    pname = t.lower()
                else:
                    pname = pname + "-" + t.lower()
                i += 1           
    elif(' ' in pname):
        # here we have to handle Cristiano Ronaldo mapping it to cristiano-ronaldo to maximize the match in the players dataframe 
        i = 0
        for t in row['short_name'].split(' '):
            if i == 0:
                pname = t.lower()
            else:
                pname = pname + "-" + t.lower()
            i += 1
    pname = strip_accents(pname)
    
    # find sim with the full name 
    fullname = row['long_name'].lower()
    i = 0
    for t in fullname.split(' '):
        if i == 0:
            fullname = t.lower()
        else:
            fullname = fullname + "-" + t.lower()
        i += 1 
    fullname = strip_accents(fullname)
    # check the players with that last name
    names =  players[players['name'].str.contains(pname)]['name']
    #find max similarity    
    maxN = 0
    playerId = ''
    for n in names:
        sim = SequenceMatcher(None, fullname, n).ratio()
        if (maxN < sim):
            maxN = sim
            playerId = players.loc[players['name'] == n].index[0]
        
    #if we get a valid playerId we can connect the Fifa stats to the transfermrkt player
    if (playerId != ''):
        #remove the row from the player dataframe to avoid futher matchings (we know data will contain errors)
        players = players.drop(index=playerId)
        idU = "player"+str(playerId)
        Player = URIRef(SO[idU])
        g.add((Player, SO['overallFifaValue'], Literal(row['overall'], datatype=XSD.int)))
        g.add((Player, SO['growthFifaPotential'], Literal(row['potential'], datatype=XSD.int)))
        g.add((Player, SO['economicValue'], Literal(row['value_eur'], datatype=XSD.int)))
        g.add((Player, SO['annualWage'], Literal(row['wage_eur'], datatype=XSD.int))) 
        
        pFeatures = str(row['player_tags'])
        if pFeatures != '_' and pFeatures != '':
            pFeatures = pFeatures.split(',')
            for feature in pFeatures:
                feature = feature.strip()
                feature = re.sub('#', '', feature)
                g.add((Player, SO['playerFeature'], Literal(feature, datatype=XSD.string)))
        
        if row['contract_valid_until'] != '_' and row['contract_valid_until'] != '':
            g.add((Player, SO['contractValidTo'], Literal(int(row['contract_valid_until']), datatype=XSD.gYear)))        

        g.add((Player, SO['birthday'], Literal(row['dob'], datatype=XSD.date)))
        g.add((Player, SO['height'], Literal(row['height_cm'], datatype=XSD.int)))
        g.add((Player, SO['weight'], Literal(row['weight_kg'], datatype=XSD.int)))
        
        
        nationality = row['nationality'] 
        nationality = nationality.replace(" ", "_")
        # create the RDF node
        Country = URIRef(CNS[nationality])
        # add the edge connecting the Movie and the Country 
        g.add((Player, SO['nationality'], Country))   

        # Homework: extend the code to populate the 'propertyOf' edge
        

NameError: name 'players' is not defined

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'players.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: user 1.51 s, sys: 8.25 ms, total: 1.52 s
Wall time: 1.52 s


## Jobs

In [ ]:
# Load the CSV files in memory
apps = pd.read_csv(appearancesUrl, sep=',', index_col='appearance_id', keep_default_na=False, na_values=['_'])
games = pd.read_csv(gamesUrl, sep=',', index_col='game_id', keep_default_na=False, na_values=['_'])

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("so", SO)

In [ ]:
#iterate over the games dataframe
for index, row in games.iterrows():
    # we use the transfermrket URL as URI
    Game = URIRef(row['url'])
    g.add((Game, RDF.type, SO.Game))
    idU1 = "club"+str(row['home_club_id'])
    idU2 = "club"+str(row['away_club_id'])
    HomeClub = URIRef(SO[idU1])
    AwayClub = URIRef(SO[idU2])
    g.add((Game, SO['homeClub'], HomeClub))
    g.add((Game, SO['awayClub'], AwayClub))    
    g.add((Game, SO['matchDay'], Literal(row['date'], datatype=XSD.date)))
    g.add((Game, SO['homeClubGoals'], Literal(row['home_club_goals'], datatype=XSD.int)))
    g.add((Game, SO['awayClubGoals'], Literal(row['away_club_goals'], datatype=XSD.int)))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'games.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: user 404 ms, sys: 3.27 ms, total: 407 ms
Wall time: 409 ms


In [ ]:
# reload the full players dataframe
players = pd.read_csv(playersUrl, sep=',', index_col='player_id', keep_default_na=False, na_values=['_'])
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("so", SO)

In [ ]:
oldgameid = ''
for index, row in apps.iterrows():
    idA = "appearance"+str(index)
    idP = "player"+str(row['player_id'])
    Appearance = URIRef(SO[idA])
    Player = URIRef(SO[idP])
    currgameid = str(row['game_id'])
    idG = "game"+currgameid
    Game = URIRef(SO[idG])
    g.add((Appearance, RDF.type, SO.Appearance))
    g.add((Player, SO['appearIn'], Appearance))
    g.add((Appearance, SO['playIn'], Game))

    g.add((Appearance, SO['goals'], Literal(row['goals'], datatype=XSD.int)))
    g.add((Appearance, SO['assists'], Literal(row['assists'], datatype=XSD.int)))
    g.add((Appearance, SO['minutesPlayed'], Literal(row['minutes_played'], datatype=XSD.int)))
    g.add((Appearance, SO['yellowCard'], Literal(row['yellow_cards'], datatype=XSD.int)))
    g.add((Appearance, SO['redCard'], Literal(row['red_cards'], datatype=XSD.int)))

    #add this triple only once per game
    if (currgameid != oldgameid):
        idL = "league"+str(row['league_id'])
        g.add((Game, SO['belongTo'], URIRef(SO[idL])))
        oldgameid = currgameid

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'appearances.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: user 12.2 s, sys: 48.5 ms, total: 12.3 s
Wall time: 12.3 s
